In [18]:
from datetime import datetime

class Forum:
    messages = []

    def init(self):
        self.messages = []

    def recupDate(self):
        dateBrute = str(datetime.now())
        date = dateBrute[11:13] + "h" + dateBrute[14:16] + "m" + dateBrute[17:19] + "s"
        return date

    def calculDate(self, date2):
            msg_date = list(date2)
            date = ""
            i=0
            while msg_date[i].isdigit() :
                date += msg_date[i]
                i+=1
            heure = int(date[0:2])
            date = ""
            i += 1
            while msg_date[i].isdigit() :
                date += msg_date[i]
                i+=1
            minute = int(date[0:2])
            date = ""
            i += 1
            while len(msg_date) and msg_date[i].isdigit() :
                date += msg_date[i]
                i+=1
            seconde = int(date[0:2])
            total = 3600*heure+60*minute+seconde
            return (total)
    
    def recupererMessagesAnonymes(self, date_debut, date_fin) :
        for msg in self.messages :
            date = self.calculDate(msg[1])
            date_start = self.calculDate(date_debut)
            date_end = self.calculDate(date_fin)
            if date > date_start and date < date_end :
                print(msg)

    def posterMessageAnonyme(self, message) :
        date = self.recupDate()
        self.messages.append([message, date])



In [19]:
if __name__ == "__main__" :
    print("start")
    forum = Forum()
    while(True) :
        string = input()
        if "show_message" == string :
            print("entrez la date au format 11h02m03s" )
            date_debut = input()
            date_fin = input()
            forum.recupererMessagesAnonymes(date_debut, date_fin)
        else :
            forum.posterMessageAnonyme(string)

start


KeyboardInterrupt: Interrupted by user

In [26]:
import time
import random

class User :
    name = ""

    def __init__(self, nom) :
        self.name = nom

    def genererSecret(self, forum, nom1, nom2, duree) :
        while duree > 0 :
            print(duree)
            attente = random.randint(0, 2)
            time.sleep(attente)
            bit = random.randint(0, 1)
            if bit == 0 :
                message = nom1
            else :
                message = nom2
            forum.posterMessageAnonyme(message)
            duree -= attente


if __name__ == "__main__" :
    print("start")
    forum = Forum()
    bob = User("bob")
    alice = User("alice")
    time.sleep(1)
    bob.genererSecret(forum, bob, alice, 20)
    alice.genererSecret(forum, bob, alice, 20)
    time.sleep(2)
    forum.recupererMessagesAnonymes("08l08l08l", "10h10h10h")
    

start
20
19
17
15
15
14
14
13
12
10
10
8
6
6
4
3
3
3
3
3
1
20
20
20
18
18
18
17
16
16
16
16
14
13
11
11
10
9
8
6
5
3
1
[<__main__.User object at 0x0000021273439C70>, '09h09m20s']
[<__main__.User object at 0x00000212734395B0>, '09h09m21s']
[<__main__.User object at 0x00000212734395B0>, '09h09m21s']
[<__main__.User object at 0x0000021273439C70>, '09h09m23s']
[<__main__.User object at 0x0000021273439C70>, '09h09m24s']
[<__main__.User object at 0x0000021273439C70>, '09h09m24s']
[<__main__.User object at 0x00000212734395B0>, '09h09m26s']
[<__main__.User object at 0x0000021273439C70>, '09h09m27s']
[<__main__.User object at 0x00000212734395B0>, '09h09m28s']
[<__main__.User object at 0x0000021273439C70>, '09h09m28s']
[<__main__.User object at 0x0000021273439C70>, '09h09m29s']
[<__main__.User object at 0x0000021273439C70>, '09h09m29s']
[<__main__.User object at 0x00000212734395B0>, '09h09m31s']
[<__main__.User object at 0x00000212734395B0>, '09h09m31s']
[<__main__.User object at 0x0000021273439

# <b> Canal de diffusion anonyme et génération de secret </b>

<i> Compte-rendu du TP2 par Valentin SÉRÉE et Jérémy SERENA [INFO3]</i>